### 1.安裝virtualbox 
選擇window hosts
> https://www.virtualbox.org/wiki/Downloads:


### 2.下載作業系統 
這邊使用linux的CentOS 7
選CentOS-7-x86_64-DVD-2009.iso 即可
>http://centos.cs.nctu.edu.tw/7.9.2009/isos/x86_64/

### 3.開啟virtualbox
按下新增
>設定主機名稱  
類型:linux  
版本:Red Hat-64bit  
RAM設定:2048  
硬碟大小設定:30G  
  
按下設定
>存放裝置-加入光碟機
加入-選擇剛剛下載的.iso檔  
確認

按下啟動
>  
選擇 install CentOS7  
選擇 中文  
選擇軟體 看各人所需    
新增root密碼 (要記住)  
安裝-重新啟動  
輸入帳號:root & 密碼:

### 4.開啟網卡
先輸入 yum --help查看yum有沒有動作
>[root@localhost ~] yum --help  

輸入cd .. 
輸入ls 尋找etc
>[root@localhost ~] cd ..
>[root@localhost ~] ls

輸入cd etc 進入
>[root@localhost etc] cd etc

輸入ls 尋找resolv.conf  
輸入vim resolv.conf 進入編輯
>[root@localhost etc] ls
>[root@localhost etc] vim resolv.conf

打上
>nameserver 8.8.8.8  
search localdomain

按下ESC 輸入:wq退出

驗證網卡配置，進入network-scripts找到ifcfg-ens33網卡配置文件  
若安裝桌面版，檔名可能是ifcfg-enp0s3
>[root@localhost ~] cd /etc/sysconfig/network-scripts  
[root@localhost network-scripts] ls -a 
[root@localhost network-scripts] vim ifcfg-ens33   

按i進入編輯，將onboot的值改為yes,再按esc後輸入:wq關閉文件  

重新啟動CentOS
>[root@localhost network-scripts] reboot

驗證yum是否正常運作
[root@localhost ~] yum provides ifconfig

### 4.安裝MySQL
安裝MySQL Yum repository
>[root@localhost ~] wget https://repo.mysql.com//mysql57-community-release-el7-11.noarch.rpm  
[root@localhost ~] yum localinstall mysql57-community-release-el7-11.noarch.rpm

確認yum repository已經安裝
>[root@localhost ~] yum repolist enabled | grep "mysql.*-community.*"

查看MySQL發行版本
>[root@localhost ~] yum repolist all | grep mysql

安裝MySQL 
>[root@localhost ~] yum install mysql-community-server

確認安裝完成後啟動服務命令如下，並可查詢啟動狀態
>[root@localhost ~] systemctl start mysqld
[root@localhost ~] systemctl status mysqld

MySQL在安裝後會為root產生一組隨機密碼放在/var/log/mysqld.log中。  
查看臨時生成的帳號密碼的方式如下:
>[root@localhost ~] grep 'temporary password' /var/log/mysqld.log

若需要修改隨機密碼可用下列SQL語法修改為自定義密碼
>[root@localhost ~] mysql -uroot -p //登入MySQL 輸入帳號root 密碼為剛剛.log中所查  
>>mysql> ALTER USER 'root'@'localhost' IDENTIFIED BY 'NewPassWord!'; //' '中為你所更改密碼

注意: 密碼須包含一個大寫字母、一個小寫字母、一個數字和一個特殊符號，且長度至少為8字元

### 5.遠端連接MySQL設定
外部連接需開啟防火牆MySQL預設3306 Port  
檢視3306 Port是否開啟
>[root@localhost ~] firewall-cmd --query-port=3306/tcp  
no

目前3306為關閉狀態，需開啟外部才可連接MySQL
開啟指令
>[root@localhost ~] firewall-cmd --zone=public --add-port=3306/tcp --permanent

關閉指令
>[root@localhost ~] firewall-cmd --zone=public --remove-port=3306/tcp --permanent

設定好後重啟防火牆
>[root@localhost ~] firewall-cmd --reload

接著修改root用戶可任意IP登入

>mysql> update mysql.user set host = '%' where user = 'root';  
Query OK, 1 row affected (0.00 sec)  
Rows matched: 1  Changed: 1  Warnings: 0  

最後生效修改命令
>mysql> flush privileges;

退出mysql
>mysql> exit;

輸入ifcondig查看IP
>[root@localhost ~] ifconfig

關閉CentOS
>[root@localhost ~] shutdown now

### 6.連結workbranch
打開virtualbox  
按下設定-網路-介面卡2  
附加到選擇:橋接介面卡  
按確定  
按下啟動

輸入帳號密碼進入後再次輸入 ifconfig  
中間會多出一排enp0s8  
將enp0s8的 inet 輸入至workbranch即可連結